In [1]:
using JuMP, Gurobi
using CSV, LinearAlgebra, DataFrames

In [3]:
#load social and ecological data and orgnize 
FL = repeat([Float64], inner=15)
dtype = append!([String], FL);

regional_EF = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/social/new_EF_SC1.csv",header=1,delim=",", types=dtype) |> DataFrame    
capacity = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/social/capacity2.csv",header=1,delim=",", types=dtype) |> DataFrame    
distance = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/social/distance.csv",header=1,delim=",") |> DataFrame 
LCA_model = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/social/LCA_model2.csv",header=1,delim=",") |> DataFrame 
D_Dsoc = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/social/D_Dsoc1.csv",header=1,delim=",") |> DataFrame
GDP = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/social/GDP.csv",header=1,delim=",") |> DataFrame;
emi_sink = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/SC_regional/emission_sink1.csv",header=1,delim=",") |> DataFrame;

In [11]:
cell_demand = 0.001*164.98*(1.36*1e6)*3           # annual demand of Li battery for tesla (1.369M EV/yr, ~2 NMC111 pack/EV, 226 kg/pack (35kwh/pack), 80~100 kWh per EV)

D = D_Dsoc[!, "D"]          # national CO2 emission (ton/yr)
Dsoc = D_Dsoc[!, "Dsoc ton/yr"]           # national CO2 sink (ton/yr)

EF_trans = 1.005/10000                        # ton CO2/km*ton (The average freight truck in the U.S. emits 161.8 grams of CO2 per ton-mile)
process = LCA_model[!,"process"]
countries = capacity[!,"country"]
ncty = size(countries,1)                          # No. of countries
nproc = size(process,1);                          # No. of processes 

mkt_loc = findfirst(isequal("United States"), countries)
mkt_proc = findfirst(isequal("battery"), process)

# seperate model
cathode = 1:4
cell = 5:10
noncell = [12,13]
battery = [11,14]
scaler = LCA_model[!,"scaler"]
price = LCA_model[!,"price (usd/ton product)"]
vGDP = GDP[!,"GDP usd"];

In [12]:
up_cath = scaler[1:4] * scaler[5] * scaler[11]
up_cell = scaler[5:10] * scaler[11]
cell_sef = scaler[11]
up_noncell = scaler[12:13] * scaler[14]
noncell_sef = scaler[14]
battery_sef = scaler[15]

1.0

In [13]:
input_amount = []
input_amount = vcat(up_cath, up_cell, cell_sef, up_noncell, noncell_sef, battery_sef) .* cell_demand
input_amount  # ton

15-element Vector{Float64}:
  74015.62808111998
 103583.22885504
 100877.69675807998
 103389.97656239998
 193252.29264
 107850.39563999999
   7808.173439999999
   1952.0433599999997
  46361.02979999999
  89793.99455999998
 488010.8399999999
   3365.5919999999996
   2692.4736
 673118.3999999999
 673118.3999999999

In [14]:
MktV = price .* input_amount 

15-element Vector{Float64}:
 2.960625123244799e9
 8.234866693975679e9
 1.5232532210470077e8
 2.0812402282011118e9
 1.430066965536e9
 2.1570079127999997e9
 9.432273515519999e6
 2.2311855604799995e6
 3.8062405465799993e8
 1.9188976637471995e8
 1.3371497015999998e10
 4.543549199999999e6
 1.61548416e9
 2.3626455839999995e9
 1.5750970559999998e10

In [15]:
maximum(MktV)

1.5750970559999998e10

In [16]:
minimum(vGDP)

5.54e10

----

In [17]:
model = Model(Gurobi.Optimizer);

Academic license - for non-commercial use only - expires 2023-11-27


In [18]:
#variables 
@variable(model, x[1:ncty, 1:nproc] >= 0)            # x[i,k] production amount of product k at location i
@variable(model, y[1:ncty, 1:ncty, 1:nproc] >= 0)   # y[i,j,k] ship product k from i to j 
@variable(model, delta[1:ncty], binary=true);

In [19]:
# capacity constraint: x[i,k] <= capacity[i][k]
for k in 1:nproc
    for i in 1:ncty
        @constraint(model, x[i,k] <= capacity[!, 2:end][i,k])
    end
end

In [20]:
# node output flow constraint
for k in 1:nproc
    for i in 1:ncty
        @constraint(model, sum(y[i,j,k] for j in 1:ncty) == x[i,k])
    end
end

In [21]:
# node output flow to market y[i,38, 15]
for i in 1:ncty
    for j in [e for e in 1:ncty if e != mkt_loc]
        @constraint(model, y[i,j,mkt_proc] == 0)
    end
end

In [22]:
for k in 1:nproc-1
    @constraint(model, sum(x[i,k] for i in 1:ncty) == input_amount[k])
end

In [23]:
# final demand constraint
@constraint(model, sum(y[i,mkt_loc,mkt_proc] for i in 1:ncty) == cell_demand);

In [24]:
# cathode LCA constraints (index=5)
for k in cathode
    for j in 1:ncty
        @constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,5] * scaler[k])
    end
end


# cell LCA constraints (index=11)
for k in cell
    for j in 1:ncty
        @constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,11] * scaler[k])
    end
end


# non cell LCA constraints (index=14)
for k in noncell
    for j in 1:ncty
        @constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,14] * scaler[k])
    end
end


# battery LCA constraints (index=15)
for k in battery
    for j in 1:ncty
        @constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,15] * scaler[k])
    end
end


In [25]:
Pemi = 0
Pincome = 0
proD = Vector{AffExpr}(undef, ncty)
proM = Vector{AffExpr}(undef, ncty)

for i in 1:ncty
    for k in 1:nproc
        Pemi += regional_EF[i,k+1] * x[i,k]
        Pincome += x[i,k] * price[k]
    end
    proD[i] = Pemi
    proM[i] = Pincome
end


Temi = 0
transD = Vector{AffExpr}(undef, ncty)
for i in 1:ncty
    for j in 1:ncty
        amount = sum(y[i,j,k] for k in 1:nproc)
        Temi += amount * distance[!, 2:end][i,j] * EF_trans
    end
    transD[i] = Temi
end

In [26]:
proM[1]

40000 x[1,1] + 79500 x[1,2] + 1510 x[1,3] + 20130 x[1,4] + 7400 x[1,5] + 20000 x[1,6] + 1208 x[1,7] + 1143 x[1,8] + 8210 x[1,9] + 2137 x[1,10] + 27400 x[1,11] + 1350 x[1,12] + 600000 x[1,13] + 3510 x[1,14] + 23400 x[1,15]

In [28]:
for i in 1:ncty
    @constraint(model, proM[i] <= vGDP[i])
end

In [29]:
M = 1e20
Allo_soc = proM ./ vGDP .* Dsoc
SJ = proD + transD - Allo_soc;

In [30]:
for i in 1:ncty
    @constraint(model, SJ[i] >= (delta[i] - 1) * M)
end

for i in 1:ncty
    @constraint(model, SJ[i] <= delta[i] * M)
end

In [40]:
@objective(model, Max, sum(delta));

In [41]:
JuMP.optimize!(model)

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 2325 rows, 13980 columns and 461250 nonzeros
Model fingerprint: 0xb45085d4
Variable types: 13950 continuous, 30 integer (30 binary)
Coefficient statistics:
  Matrix range     [2e-04, 1e+20]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 1e+20]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.

MIP start from previous solve produced solution with objective 24 (0.10s)
Loaded MIP start from previous solve with objective 24

Presolve removed 2233 rows and 13772 columns
Presolve time: 0.08s
Presolved: 92 rows, 208 columns, 495 nonzeros
Found heuristic solution: objective 30.0000000
Variable types: 208 continuous, 0 integer (0 binary)

Explored 0 nodes (0 simplex iterations) in 0.27 seconds
Thread count was 12 (of 12 available processo

In [42]:
JuMP.value.(x)

30×15 Matrix{Float64}:
     0.0      0.0  0.0            0.0  …     0.0   0.0        0.0
 11415.6      0.0  0.0            0.0        0.0   0.0        0.0
  2200.0      0.0  0.0            0.0        0.0   0.0        0.0
   500.0      0.0  0.0            0.0        0.0   0.0        0.0
 39000.0      0.0  0.0            0.0        0.0   0.0        0.0
 19000.0      0.0  0.0            0.0  …     0.0   0.0        0.0
     0.0      0.0  0.0            0.0        0.0   0.0        0.0
     0.0  82929.2  0.0            0.0        0.0   0.0        0.0
     0.0      0.0  0.0            0.0        0.0   0.0        0.0
     0.0      0.0  0.0            0.0        0.0   0.0        0.0
     0.0      0.0  0.0            0.0  …     0.0   0.0        0.0
     0.0      0.0  0.0            0.0        0.0   0.0        0.0
     0.0   9454.0  0.0            0.0        0.0   0.0        0.0
     ⋮                                 ⋱                      
     0.0      0.0  0.0            0.0        0.0   0.0  

In [27]:
optx = copy(JuMP.value.(x))
opty = copy(JuMP.value.(y));

In [28]:
res_x = DataFrame(optx, :auto)
rename!(res_x, ["x$i" => proc for (i, proc) in enumerate(process)])
insertcols!(res_x, 1, :country => countries);

In [59]:
# DataFrame(opty[1:end, 1:end, 1], :auto)

In [29]:
CSV.write("/Users/bourg/Desktop/x_matrix_0915_worst.csv", res_x) 

"/Users/bourg/Desktop/x_matrix_0915_worst.csv"

----

In [37]:
Pemi = 0
Pincome = 0
proD = zeros(ncty)
proM = zeros(ncty)

for i in 1:ncty
    for k in 1:nproc
        Pemi += regional_EF[i,k+1] * optx[i,k]
        Pincome += optx[i,k] * price[k]
    end
    proD[i] = Pemi
    proM[i] = Pincome
end

In [39]:
Temi = 0
transD = zeros(ncty)
for i in 1:ncty
    for j in 1:ncty
        amount = sum(opty[i,j,k] for k in 1:nproc)
        Temi += amount * distance[!, 2:end][i,j] * EF_trans
    end
    transD[i] = Temi
end

In [41]:
Allo_soc = proM ./ vGDP .* Dsoc;

In [42]:
SJ = proD + transD - Allo_soc

30-element Vector{Float64}:
       0.0
  146302.13332199663
  123831.42101741045
  206256.08675930544
  159912.63574448245
       1.056562300404097e6
  875207.1441314534
 -794987.1116928686
       2.924519861638376e6
       3.0915770757717355e6
       3.950255987924516e6
  615154.6924511166
       3.1179878368311003e6
       ⋮
       5.53059013545199e6
       4.554944546771766e6
       3.2063561278521023e6
       5.516343264027968e6
      -9.0869063574939e8
      -3.938191876841539e8
       4.066881103618127e6
      -5.091918499342205e8
       6.573037055059096e6
       5.237567647721427e6
       7.624298300202082e6
       1.0146016860700918e7

In [45]:
optdelta = JuMP.value.(delta);

In [49]:
SJ[2]

146302.13332199663

In [48]:
(optdelta[2] - 1) * M

-1.0e20

In [31]:
# using Tables
# res = Tables.table(optx[1:end, 1:end]);
# CSV.write("/Users/bourg/Desktop/x_matrix_regional0914.csv", res) 

In [30]:
# res_y02 = Tables.table(opty[1:end, 1:end, 02])
# CSV.write("/Users/bourg/Desktop/y02_matrix.csv", res_y02) 